# Asynchronous Programming

## Background
*Note that this tutorial assumes basic understanding of Runhouse Functions & Modules; it is recommended that you check out [our functions and modules tutorial](https://www.run.house/docs/tutorials/api-modules) before diving into this one.* 

As we've discussed before, once you take a Python function or module and send it to a Runhouse cluster, the cluster holds your resource in memory, and each time that function or module is called by a client, it simply accesses it in memory and calls it. Under the hood, we have a fully asynchronous server running (FastAPI), and a separate process for each environment where your Runhouse resources live. These processes all have their own async event loops, and if you run synchronous functions on Runhouse, they are ran in a separate thread to allow for many concurrent calls to the cluster. **Note that if you are unfamiliar with asynchronous programming in Python, you should just continue using standard, Python sync functions and leave the performance to us**.

## Native Async Functions

But, what if you're writing code that leverages Python's powerful asynchronous functionality? Luckily, we provide rich async support in a variety of ways. First off, any function that is labeled with Python's async keyword will be *executed within the async event loop*, and not within a separate thread. **This means that you should be very careful that you are not running any costly, synchronous code within an async function, to avoid blocking up your the event loop within your environment on the server. Poorly written async functions will not block the entire Runhouse daemon, but will block other functions within the same environment as the user code.** 

Client side, you also need to `await` a call to this function the same way you would if the function was running locally. Let's check out an example. First, we'll start a local Runhouse daemon to mess with:

In [ ]:
! runhouse restart

Then, we'll define a simple `async` function to send to Runhouse:

In [9]:
async def async_test(time_to_sleep: int):
    import asyncio
    
    await asyncio.sleep(time_to_sleep)
    return time_to_sleep

We can send this to Runhouse the same way we would any other Runhouse function:

In [10]:
import runhouse as rh

async_test_fn_remote = rh.function(async_test).to(rh.here)

INFO | 2024-04-30 18:50:35.023995 | Writing out function to /Users/rohinbhasin/work/notebooks/docs/async_test_fn.py. Please make sure the function does not rely on any local variables, including imports (which should be moved inside the function body).
INFO | 2024-04-30 18:50:35.060478 | Sending module async_test of type <class 'runhouse.resources.functions.function.Function'> to local Runhouse daemon


Then, we can call this function as we would if it were a local async function. The network call to the remote cluster will execute asynchronously within our local event loop (our code backed by `httpx.AsyncClient`) and the async function itself will execute within the async event loop on the remote server.

In [11]:
await async_test_fn_remote(2)

2

Voila! Async functions are supported the way you'd expect them to be. There are a few other advanced cases, though:

## Advanced: Running Sync Functions as Async Locally

There's another important case that we support. Now that your synchronous functions are running on a remote Runhouse machine, there is network I/O involved in making calls to these functions. You may want to run these functions asynchronously remotely, so that you can not block your Python event loop locally with a network call. Note that this means your local code will have to be using async primitives even though it is calling what you defined as a sync function. Let's check out an example of this:

In [14]:
def synchronous_sleep(time_to_sleep: int):
    import time

    time.sleep(time_to_sleep)
    return time_to_sleep

sync_sleep_fn_remote = rh.function(synchronous_sleep).to(rh.here)

INFO | 2024-04-30 18:57:00.533012 | Writing out function to /Users/rohinbhasin/work/notebooks/docs/synchronous_sleep_fn.py. Please make sure the function does not rely on any local variables, including imports (which should be moved inside the function body).
INFO | 2024-04-30 18:57:00.577673 | Sending module synchronous_sleep of type <class 'runhouse.resources.functions.function.Function'> to local Runhouse daemon


We can now call this function with the `run_async` argument set to to `True`. This makes it not actually run locally immediately, and instead returns a coroutine that you'd await, as if this function were asynchronous. Note that in this case, the functions runs as a synchronous function remotely (in a thread), but locally behaves as an async function.

In [15]:
await sync_sleep_fn_remote(2, run_async=True)

2

If I wanted, I could still call this function as a fully synchronous function:

In [16]:
sync_sleep_fn_remote(2)

2

## Advanced: Running Async Functions as Sync Locally

The third critical case that we support is mostly applicable when you're writing async code for the purpose of running it on the Runhouse cluster, but want to make synchronous calls to the server. The reason for you writing async code to run on the server is because our Runhouse server uses ASGI and runs everything asynchronously, so you can take advantage of the performance gains that come along with async code, but call it locally as you would a normal client calling a normal server, unaware of the backend implementation of the server. We can take the same async function I defined earlier and call it synchronously:

In [17]:
async_test_fn_remote(2, run_async=False)

2

That's all there is to it! We've tried our hardest to make working with async code seamless from a user's perspective. There are other edge cases we've put time into supporting and we're happy to discuss architecture anytime -- feel free to file an issue or join us on Discord to discuss more!